In [1]:
import torch
import torchaudio
import torch.nn as nn

In [2]:
import torchaudio
from torchaudio.utils import download_asset
from IPython.display import Audio

In [3]:
SAMPLE_SONG = download_asset("tutorial-assets/hdemucs_mix.wav")
waveform, sample_rate = torchaudio.load(SAMPLE_SONG)  

In [40]:
waveform = waveform[:44100*1,:44100*5]

In [4]:
from loss import MultiResSpecLoss, Stft

In [5]:
criterion_2 = MultiResSpecLoss(factor = 500,
                               f_complex = 500,
                               gamma = 0.3,
                               n_ffts = [256, 4096])

In [6]:
criterion_2(waveform[None], waveform[None]+1)

tensor(24.3652)

In [5]:
import lpips
loss_fn = lpips.LPIPS(net='vgg')

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


C:\Users\Максим\venv_vk\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Максим\venv_vk\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: C:\Users\Максим\venv_vk\lib\site-packages\lpips\weights\v0.1\vgg.pth


In [7]:
for param in loss_fn.parameters():
    param.requires_grad = False

In [6]:
loss_fn()

['L',
 'T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_regist

In [ ]:
#!pip install lpips

In [12]:
from torch import Tensor

In [47]:
class PerceptualLoss(nn.Module):
    def __init__(self, n_fft, net='vgg'):
        super().__init__()
        self.stft = Stft(n_fft)
        self.loss = lpips.LPIPS(net=net)
        for param in self.loss.parameters():
            param.requires_grad = False
            
    def _reshape_source(self, x, source):
        B, S, C, Fr, T = x.shape
        x = torch.concat([x[:,sourc], x[:,sourc].mean(dim=1).view(B,1,Fr,T)], dim=1)
        return x
    
    def forward(self, inputs: Tensor, target: Tensor) -> Tensor:
        input_spec = self.stft(inputs)
        target_spec = self.stft(target)
        drums = self.loss(self._reshape_source(input_spec.abs(), 0), self._reshape_source(target_spec.abs(), 0))
        bass = self.loss(self._reshape_source(input_spec.abs(), 1), self._reshape_source(target_spec.abs(), 1))
        other = self.loss(self._reshape_source(input_spec.abs(), 2), self._reshape_source(target_spec.abs(), 2))
        vocals = self.loss(self._reshape_source(input_spec.abs(), 3), self._reshape_source(target_spec.abs(), 3))
        
        return drums.sum(), bass.sum(), other.sum(), vocals.sum()

In [48]:
loss = PerceptualLoss(n_fft = 4096)


Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: C:\Users\Максим\venv_vk\lib\site-packages\lpips\weights\v0.1\vgg.pth


In [49]:
data = torch.concat( [waveform[None],waveform[None],waveform[None]], dim=0)


In [50]:
loss(data,data).shape

torch.Size([3, 2049, 216])

In [51]:
data.shape

torch.Size([3, 220500])

In [38]:
waveform[None].shape

torch.Size([1, 2, 7560512])

In [54]:
from wav import get_datasets
import augment
import distrib

ModuleNotFoundError: No module named 'wav'

In [63]:
from model.wav import get_datasets

ImportError: attempted relative import with no known parent package

In [62]:
ls

 ’®¬ ў гбва®©бвўҐ C Ё¬ҐҐв ¬ҐвЄг OS
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : 4AD1-C5F7

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ C:\Users\Њ ЄбЁ¬\venv_vk\VK_PROJECT\losses

12.04.2023  21:45    <DIR>          .
12.04.2023  21:45    <DIR>          ..
12.04.2023  21:13    <DIR>          .ipynb_checkpoints
12.04.2023  21:23    <DIR>          __pycache__
12.04.2023  21:12             2я647 loss.py
31.03.2023  23:32    <DIR>          musdb18hq
12.04.2023  21:45             6я408 Test.ipynb
12.04.2023  21:18               574 utils.py
               3 д ©«®ў          9я629 Ў ©в
               5 Ї Ї®Є  21я124я476я928 Ў ©в бў®Ў®¤­®
